In [6]:
! pip install confluent-kafka requests grpcio protobuf grpcio-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 32.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 54.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.6/319.6 kB 35.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.3
    Uninstalling protobuf-4.24.3:
      Successfully uninstalled protobuf-4.24.3


In [9]:
import os
import requests
import tempfile
import importlib.util
from google.protobuf.descriptor_pb2 import FileDescriptorSet
from google.protobuf import descriptor_pb2
from google.protobuf.message_factory import GetMessageClass
from google.protobuf.descriptor_pool import DescriptorPool
from google.protobuf.descriptor_pb2 import FileDescriptorProto
from confluent_kafka import Consumer

# Configuration
KAFKA_BROKER = "redpanda:9092"
SCHEMA_REGISTRY_URL = "http://redpanda:8081"
TOPIC_NAME = "search_requests"
GROUP_ID = "search-consumer-2"

# Kafka Consumer Config
consumer = Consumer({
    "bootstrap.servers": KAFKA_BROKER,
    "group.id": GROUP_ID,
    "auto.offset.reset": "earliest",
})

consumer.subscribe([TOPIC_NAME])


def fetch_schema_from_registry():
    """Fetches the latest Protobuf schema from the Redpanda Schema Registry."""
    schema_subject = f"{TOPIC_NAME}-value"
    schema_url = f"{SCHEMA_REGISTRY_URL}/subjects/{schema_subject}/versions/latest"
    
    response = requests.get(schema_url)
    if response.status_code == 200:
        schema_data = response.json()
        return schema_data["schema"]
    else:
        raise Exception(f"Failed to fetch schema: {response.text}")


def compile_proto_from_schema(proto_schema):
    """Compiles the Protobuf schema dynamically and loads the message class."""
    with tempfile.TemporaryDirectory() as temp_dir:
        proto_file_path = os.path.join(temp_dir, "dynamic.proto")
        
        # Write the schema to a .proto file
        with open(proto_file_path, "w") as f:
            f.write(proto_schema)

        # Compile the .proto file
        os.system(f"protoc --python_out={temp_dir} --proto_path={temp_dir} {proto_file_path}")

        # Import the compiled Python module dynamically
        generated_file = os.path.join(temp_dir, "dynamic_pb2.py")
        spec = importlib.util.spec_from_file_location("dynamic_pb2", generated_file)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)

        # Return the dynamically loaded Protobuf class (assuming first message is the one we need)
        message_class_name = list(module.DESCRIPTOR.message_types_by_name.keys())[0]
        return getattr(module, message_class_name)


# Fetch schema and compile
proto_schema = fetch_schema_from_registry()
DynamicMessageClass = compile_proto_from_schema(proto_schema)

# Start consuming messages
print("\n🚀 Waiting for messages...\n")
while True:
    msg = consumer.poll(1.0)  # Poll for messages

    if msg is None:
        continue
    if msg.error():
        print(f"❌ Consumer error: {msg.error()}")
        continue

    # Deserialize the Protobuf message using the dynamically loaded class
    dynamic_message = DynamicMessageClass()
    dynamic_message.ParseFromString(msg.value())

    # Print the received message as a dictionary
    print(f"🔹 Received: {dynamic_message}")



🚀 Waiting for messages...

🔹 Received: query: "protobuf"
page_number: 5
results_per_page: 33

🔹 Received: query: "redpanda"
page_number: 1
results_per_page: 43

🔹 Received: query: "redpanda"
page_number: 7
results_per_page: 44

🔹 Received: query: "kafka"
page_number: 9
results_per_page: 44

🔹 Received: query: "redpanda"
page_number: 2
results_per_page: 50

🔹 Received: query: "streaming"
page_number: 10
results_per_page: 21

🔹 Received: query: "protobuf"
page_number: 7
results_per_page: 50

🔹 Received: query: "redpanda"
page_number: 9
results_per_page: 39

🔹 Received: query: "kafka"
page_number: 7
results_per_page: 18

🔹 Received: query: "protobuf"
page_number: 4
results_per_page: 11



KeyboardInterrupt: 

%4|1740105785.172|MAXPOLL|rdkafka#consumer-5| [thrd:main]: Application maximum poll interval (300000ms) exceeded by 405ms (adjust max.poll.interval.ms for long-running message processing): leaving group
